* charge sensitivity over charge offset
* variation of Ej/Ec ratio
* verification of analytic results

In [1]:
from DiSuQ.Torch import models
from DiSuQ.Torch.optimization import OrderingOptimization,PolynomialOptimization
from DiSuQ.Torch.optimization import lossTransition
from torch import tensor, float32 as float
from numpy import arange,linspace,std as deviation,mean,sqrt
from DiSuQ.utils import plotCompare
import pickle
from torch import set_num_threads
set_num_threads(32)

In [51]:
Ej = 30.02 ; Ec = 10.2
Ej = Ec*50.

In [52]:
basis = [512]; rep = 'Q'
circuit = models.transmon(basis,Ej,Ec,sparse=False)
print(circuit.circuitComponents())

{'J': 510.0, 'C': 0.04901960864663124}


In [53]:
charge_offset = linspace(-2, 2, 220) #65,17,33

In [54]:
H_LC = circuit.chargeHamiltonianLC()
H_J = circuit.josephsonCharge
ground,Ist = circuit.circuitEnergy(H_LC,H_J,dict(),grad=False)[:2]
Delta = Ist-ground

In [55]:
G,I,II,III = [],[],[],[]
for q in charge_offset:
    H_LC = circuit.chargeHamiltonianLC()
    offset = dict([(1,tensor(q))])
    H_off = circuit.chargeChargeOffset(offset)
    H_J = circuit.josephsonCharge
    ground,Ist,IInd,IIIrd = circuit.circuitEnergy(H_LC+H_off,H_J,dict(),grad=False)[:4]
    delta = Ist-ground
    #Ist -= ground; IInd -= ground; IIIrd -= ground
    G.append(ground)
    I.append(Ist)
    II.append(IInd)
    III.append(IIIrd)

In [56]:
plots = {'0':G,'I':I,'II':II}

In [57]:
plotCompare(charge_offset,plots,'Transmon-50.-Offset Charge','$n_g$','$E_m(GHz)$',export='pdf',size=(600,600))

In [33]:
410-217,sqrt(8*Ej*Ec)

(193, 203.99999999999997)

## Ej/Ec variation

In [34]:
Ej = 30.02
ratio = linspace(1,50,100,endpoint=True)

In [35]:
std = {'std0':[],'std1':[],'std2':[]}
avg = {'mean0':[],'mean1':[],'mean2':[]}
deltaHalf = []

In [36]:
for r in ratio:
    Ec = Ej/r
    circuit = models.transmon(basis,Ej,Ec,sparse=False)
    G,I,II = [],[],[]
    for q in charge_offset:
        H_LC = circuit.chargeHamiltonianLC()
        offset = dict([(1,tensor(q))])
        H_off = circuit.chargeChargeOffset(offset)
        H_J = circuit.josephsonCharge
        ground,Ist,IInd = circuit.circuitEnergy(H_LC+H_off,H_J,dict(),grad=False)[:3]
        delta = Ist-ground
        G.append(ground/delta)
        I.append(Ist/delta)
        II.append(IInd/delta)
        if q == .5:
            deltaHalf.append(Ist/delta)
    # analysis of dispersion profile
    std['std0'].append(deviation(G)) ; avg['mean0'].append(mean(G))
    std['std1'].append(deviation(I)) ; avg['mean1'].append(mean(I))
    std['std2'].append(deviation(II)) ; avg['mean2'].append(mean(II))

In [37]:
#plots = {'delta Half' : deltaHalf}
plots.update(std)
plots.update(avg)

In [42]:
plotCompare(ratio,std,'Transmon Insensitivity-Deviation','Ej/Ec','energy(GHz)',export='pdf',size=(600,600))

In [43]:
plotCompare(ratio,avg,'Transmon Insensitivity-Mean','Ej/Ec','energy(GHz)',export='pdf',size=(600,600))